In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pymongo
from IPython.display import Image
import shutil

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

which: no chromedriver in (/c/Users/sinancengiz/Anaconda3/envs/PythonData:/c/Users/sinancengiz/Anaconda3/envs/PythonData/Library/mingw-w64/bin:/c/Users/sinancengiz/Anaconda3/envs/PythonData/Library/usr/bin:/c/Users/sinancengiz/Anaconda3/envs/PythonData/Library/bin:/c/Users/sinancengiz/Anaconda3/envs/PythonData/Scripts:/c/Users/sinancengiz/Anaconda3/envs/PythonData/bin:/c/Users/sinancengiz/bin:/mingw64/bin:/usr/local/bin:/usr/bin:/usr/bin:/mingw64/bin:/usr/bin:/c/Users/sinancengiz/bin:/c/ProgramData/Oracle/Java/javapath:/c/Program Files (x86)/Intel/iCLS Client:/c/Program Files/Intel/iCLS Client:/c/WINDOWS/system32:/c/WINDOWS:/c/WINDOWS/System32/Wbem:/c/WINDOWS/System32/WindowsPowerShell/v1.0:/c/Program Files (x86)/Intel/Intel(R) Management Engine Components/DAL:/c/Program Files/Intel/Intel(R) Management Engine Components/DAL:/c/Program Files (x86)/Intel/Intel(R) Management Engine Components/IPT:/c/Program Files/Intel/Intel(R) Management Engine Components/IPT:/cmd:/c/Program Files (x86)/

In [3]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

header = soup.find_all('div', class_='content_title')
paragraf = soup.find_all('div', class_='article_teaser_body')


print(paragraf[0].text)

The spacecraft has completed its first trajectory correction maneuver.


In [6]:
news_title = header[0].text
news_p = paragraf[0].text

In [7]:
news_title

'InSight Steers Toward Mars'

In [8]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [9]:
image = soup.find_all('div', class_='img')
featured_image_url = image[0].img['src']
print(featured_image_url)

/spaceimages/images/wallpaper/PIA22326-640x350.jpg


In [10]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('div', class_='js-tweet-text-container')

In [11]:
mars_weather = results[0].p.text
print(mars_weather)

Sol 2058 (May 21, 2018), Sunny, high 4C/39F, low -71C/-95F, pressure at 7.43 hPa, daylight 05:20-17:20


In [12]:
url = 'http://space-facts.com/mars/'

tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [13]:
df = tables[0]
df.columns = ['Planet Data Type', 'Value']
df.set_index('Planet Data Type', inplace=True)
df.head()

,Value
Planet Data Type,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"


In [14]:
html_table = df.to_html(index=False)
html_table.replace('\n', '')
df.to_html('table.html')

In [15]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [16]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [17]:
# Define database and collection
db = client.craigslist_db
collection = db.items

In [18]:
# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [55]:
# mars_image = soup.find_all('div', class_='downloads')
# img_url = mars_image[0].a["href"]
# print(img_url)

In [57]:
mars_container = soup.find_all('div', class_='container')
mars_image = mars_container[0].find_all('div', class_='collapsible results')
img_url = mars_image[0].a["href"]
print(img_url)


/search/map/Mars/Viking/cerberus_enhanced


In [60]:
contents = mars_container[0].find_all('div', class_='description')
title = contents[0].h3.text
print(title)

Cerberus Hemisphere Enhanced


In [24]:
for x in range(1, 6):

    for container in mars_container:
    # Error handling
        try:
            # Identify and return title of listing
            contents = container.find_all('div', class_='content')
            title = contents[0].h2.text
             # Identify and return link to listing
            img_url = container.find_all('div', class_='downloads')
            
            img_url = mars_image[0].a["href"]

            # Run only if title, price, and link are available
            if (title and img_url):
                # Print results
                print('-------------')
                print(title)
                print(img_url)
        
               # Dictionary to be inserted as a MongoDB document
                hemisphere_image_urls = {
                    'title': title,
                    'img_url': img_url
                  }

                collection.insert_one(hemisphere_image_urls)

        except Exception as e:
            print(e)
    
browser.click_link_by_partial_text('next')

-------------
Cerberus Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it


WebDriverException: Message: chrome not reachable
  (Session info: chrome=66.0.3359.181)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.16299 x86_64)


In [ ]:
from IPython.display import Image
Image(url=img_url)

In [ ]:
import requests
import shutil
response = requests.get(img_url, stream=True)
with open('Cerberus.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)